In [ ]:
import hashlib
import base64

from Crypto import Random
from Crypto.Cipher import AES

from PIL import Image
#helper function that will assist in setting the LSB of the value we passed
def Set_LSB(value, bit):
    if bit == '0':
        value = value & 254
    else:
        value = value | 1
    return value

#The following function is for hiding data using LSB steganography method

def Hide_message(carrier, message, outfile):
    message += chr(0)
    #The following section of the code creates two image objects:
    #one of our carrier and one for the output image. For our carrier image, 
    #we change the mode to RGBA to make sure we have the four values per pixel.
    c_image = Image.open(carrier)
    c_image = c_image.convert('RGBA')

    out = Image.new(c_image.mode, c_image.size)
    pixel_list = list(c_image.getdata())
    new_array = []

    for i in range(len(message)):
        char_int = ord(message[i])
        cb = bin(char_int)[2:].zfill(8)
        pix1 = pixel_list[i*2]
        pix2 = pixel_list[(i*2)+1]
        newpix1 = []
        newpix2 = []
        #Now that everything is set up, we can start to change the values of the pixel data 
        #we iterate 4 times (for the RGBA values) and call our helper method to set the LSB. 
        #The newpix1 function will contain the first 4 bits of our 8-bit character; 
        #newpix2 will have the last 4:

        for j in range(0,4):
            newpix1.append(Set_LSB(pix1[j], cb[j]))
            newpix2.append(Set_LSB(pix2[j], cb[j+4]))

        new_array.append(tuple(newpix1))
        new_array.append(tuple(newpix2))
#All that's left to do is extend the new_array method with the remaining pixels 
#from our carrier image and then save it using the filename parameter that was
#passed in to our Hide_message function
    new_array.extend(pixel_list[len(message)*2:])

    out.putdata(new_array)
    out.save(outfile)
#padding for the input string
BS = 16
def pad(s):
    return s + (BS - len(s) % BS) * chr(BS - len(s) % BS)

def unpad(s):
    return s[0:-s[-1]]

class AESCipher:

    def __init__( self, key ):#This is for the key generation which is a length of 32 bytes
        self.bs = 32
        self.key = hashlib.sha256(key.encode()).digest()

    def encrypt( self, raw ):#An encryption function for the message
        raw = pad(raw)
        iv = Random.new().read( AES.block_size )
        cipher = AES.new( self.key, AES.MODE_CBC, iv )
        return base64.b64encode( iv + cipher.encrypt( raw ) )
    
def main():
    msg = input("Enter your message:")
    sec_key = input("Enter your encryption key:")
    img_loc = input("Enter image name along with the extension:")
    cipher = AESCipher(sec_key)#creation of instance for AESCipher 
    encrypted = cipher.encrypt(msg)#Encrypting the message
    Hide_message(img_loc,str(encrypted),'school_cipher.png')#Hiding the encrypted message in the image

#Interpretation starts here
main()